In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
import librosa
from tensorflow.keras import backend as K
import argparse
import os
import sys
from random import getrandbits

In [2]:
def VGG11():
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding='same', input_shape=(160,101,1)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2,2), strides=(2,2)),
        
        tf.keras.layers.Conv2D(128, (3,3), activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2,2), strides=(2,2)),

        tf.keras.layers.Conv2D(256, (3,3), activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(256, (3,3), activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2,2), strides=(2,2)),

        tf.keras.layers.Conv2D(512, (3,3), activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(512, (3,3), activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2,2), strides=(2,2)),

        tf.keras.layers.Conv2D(512, (3,3), activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(512, (3,3), activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2,2), strides=(2,2)),
        
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(30)
    ])
    
    return model

In [3]:
def random_onoff():                # randomly turns on or off
    return bool(getrandbits(1))


def augment_signal(y, sr, quiet=True):
    count_changes = 0
    allow_pitch, allow_dyn, allow_noise = True, True, True
    y_mod = y
    # change pitch (w/o speed)
    if (allow_pitch) and random_onoff():
        bins_per_octave = 24  # pitch increments are quarter-steps
        pitch_pm = 4  # +/- this many quarter steps
        pitch_change = pitch_pm * 2 * (np.random.uniform() - 0.5)
        if not quiet:
            print("    pitch_change = ", pitch_change)
        y_mod = librosa.effects.pitch_shift(y, sr, n_steps=pitch_change, bins_per_octave=bins_per_octave)
        count_changes += 1
    # change dynamic range
    if (allow_dyn) and random_onoff():
        dyn_change = np.random.uniform(low=0.5, high=1.1)  # change amplitude
        if not quiet:
            print("    dyn_change = ", dyn_change)
        y_mod = y_mod * dyn_change
        count_changes += 1
        # add noise
    if (allow_noise) and random_onoff():
        noise_amp = 0.005 * np.random.uniform() * np.amax(y)
        if random_onoff():
            if not quiet:
                print("    gaussian noise_amp = ", noise_amp)
            y_mod += noise_amp * np.random.normal(size=len(y))
        else:
            if not quiet:
                print("    uniform noise_amp = ", noise_amp)
            y_mod += noise_amp * np.random.uniform(size=len(y))
        count_changes += 1

    # last-ditch effort to make sure we made a change (recursive/sloppy, but...works)
    if (0 == count_changes):
        if not quiet:
            print("No changes made to signal, trying again")
        y_mod = augment_signal(y_mod, sr, quiet=quiet)

    return y_mod


def spect_loader(path, window_size=.02, window_stride=.01, window='hamming', normalize=True, max_len=101, augment=False):
    y, sr = sf.read(path)
    y_original_len = len(y)
    
    if augment:
        y = augment_signal(y, sr)
    if not len(y) == y_original_len:
        print('augmentation ruined the audio files length!!!')
        exit()
    
    try:
        n_fft = int(sr * window_size)
    except:
        print(path)

    win_length = n_fft
    hop_length = int(sr * window_stride)
    
    # STFT
    D = librosa.stft(y, n_fft=n_fft, hop_length=hop_length, win_length=win_length, window=window)
    
    spect, phase = librosa.magphase(D)
    spect = np.log1p(spect)
    real_features_len = spect.shape[1]
    
    # make all spects with the same dims
    if spect.shape[1] < max_len:
        pad = np.zeros((spect.shape[0], max_len - spect.shape[1]))
        spect = np.hstack((spect, pad))
    elif spect.shape[1] > max_len:
        spect = spect[:, :max_len]

    if spect.shape[0] < 160:
        pad = np.zeros((160 - spect.shape[0], spect.shape[1]))
        spect = np.vstack((spect, pad))
    elif spect.shape[0] > 160:
        spect = spect[:160, :]
    spect = np.resize(spect, (spect.shape[0], spect.shape[1], 1))
    #spect = torch.FloatTensor(spect) 
    
    #normalization to zero-mean and one-std
    normalize = True
    if normalize:
        mean = np.mean(spect)
        std = np.std(spect)
        if std != 0:
            spect = spect - mean
            spect = spect / std
            
    return spect, len(y), real_features_len, sr

In [4]:
def find_classes(dir):
    classes = [d for d in os.listdir(dir) if (os.path.isdir(os.path.join(dir, d)))]
    classes.sort()
    class_to_idx = {classes[i]: i for i in range(len(classes))}
    return classes, class_to_idx

AUDIO_EXTENSIONS = ['.wav', '.WAV']

def is_audio_file(filename):
    return any(filename.endswith(extension) for extension in AUDIO_EXTENSIONS)

In [5]:
def DataLoader(path, window_size, window_stride, window_type, normalize, max_len):
    classes, class_to_idx = find_classes(path)

    spects = []
    root_dir = os.path.expanduser(path)
    count = 0

    for data in sorted(os.listdir(root_dir)):
        d = os.path.join(root_dir, data)
        if not os.path.isdir(d):
            continue
        for root, _, fnames in sorted(os.walk(d)):
            for fname in sorted(fnames):
                if is_audio_file(fname):
                    path = os.path.join(root, fname)
                    label = os.path.join(root, fname.replace(".wav", ".wrd"))
                    item = (path, class_to_idx[data], label)
                    spects.append(item)
                    count += 1

    if len(spects) == 0:
        raise (RuntimeError("Found 0 sound files in subfolders of: " + path + "Supported audio file extensions are: " + ",".join(AUDIO_EXTENSIONS)))

    dataset = []

    for index in range(np.shape(spects)[0]):
        data_dir, target, label_path = spects[index]
        spect, _, _, _ = spect_loader(data_dir, window_size, window_stride, window_type, normalize, max_len)
        dataset.append((spect, target))
    
    features = np.zeros([np.shape(dataset)[0]] + list(np.shape(dataset[0][0])))
    labels = np.zeros([np.shape(dataset)[0],30])
    
    for i in range(np.shape(dataset)[0]):
        labels[i,dataset[i][1]] = 1
        features[i,:,:,:] = dataset[i][0]
    
    return features, labels

In [6]:
# Training settings
parser = argparse.ArgumentParser(
    description='ConvNets for Speech Commands Recognition')
parser.add_argument('--train_path', default='gcommand_toy_example/train',
                    help='path to the train data folder')
parser.add_argument('--test_path', default='gcommand_toy_example/test',
                    help='path to the test data folder')
parser.add_argument('--valid_path', default='gcommand_toy_example/valid',
                    help='path to the valid data folder')
parser.add_argument('--batch_size', type=int, default=32,
                    metavar='N', help='training and valid batch size')
parser.add_argument('--test_batch_size', type=int, default=100,
                    metavar='N', help='batch size for testing')
parser.add_argument('--arc', default='VGG11',
                    help='network architecture: VGG11, VGG13, VGG16, VGG19')
parser.add_argument('--epochs', type=int, default=100,
                    metavar='N', help='number of epochs to train')
parser.add_argument('--lr', type=float, default=0.001,
                    metavar='LR', help='learning rate')
parser.add_argument('--momentum', type=float, default=0.9,
                    metavar='M', help='SGD momentum, for SGD only')
parser.add_argument('--optimizer', default='adam',
                    help='optimization method: sgd | adam')
parser.add_argument('--cuda', default=True, help='enable CUDA')
parser.add_argument('--seed', type=int, default=1234,
                    metavar='S', help='random seed')
parser.add_argument('--log-interval', type=int, default=100, metavar='N',
                    help='num of batches to wait until logging train status')
parser.add_argument('--patience', type=int, default=5, metavar='N',
                    help='how many epochs of no loss improvement should we wait before stop training')

# feature extraction options
parser.add_argument('--max_len', type=int, default=101,
                    help='window size for the stft')
parser.add_argument('--window_size', default=.02,
                    help='window size for the stft')
parser.add_argument('--window_stride', default=.01,
                    help='window stride for the stft')
parser.add_argument('--window_type', default='hamming',
                    help='window type for the stft')
parser.add_argument('--normalize', default=True,
                    help='boolean, wheather or not to normalize the spect')
parser.add_argument('--save_folder', type=str,  default='gcommand_pretraining_model/',
                    help='path to save the final model')
parser.add_argument('--class_num', type=int,  default=30,
                    help='number of classes to classify')

args = parser.parse_args(args=[])
print(args)

Namespace(arc='VGG11', batch_size=32, class_num=30, cuda=True, epochs=100, log_interval=100, lr=0.001, max_len=101, momentum=0.9, normalize=True, optimizer='adam', patience=5, save_folder='gcommand_pretraining_model/', seed=1234, test_batch_size=100, test_path='gcommand_toy_example/test', train_path='gcommand_toy_example/train', valid_path='gcommand_toy_example/valid', window_size=0.02, window_stride=0.01, window_type='hamming')


In [7]:
data_train, labels_train = DataLoader(args.train_path, window_size=args.window_size, window_stride=args.window_stride, 
                           window_type=args.window_type, normalize=args.normalize, max_len=args.max_len)

data_val, labels_val = DataLoader(args.valid_path, window_size=args.window_size, window_stride=args.window_stride, 
                           window_type=args.window_type, normalize=args.normalize, max_len=args.max_len)

In [ ]:
clf = VGG11()
clf.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), metrics=['accuracy'])
clf_train = clf.fit(data_train, labels_train, validation_data=(data_val, labels_val), epochs=1, batch_size=1, shuffle=True)
scores = clf.evaluate(data_val, labels_val)
print("Accuracy: %.2f%%" %(scores[1]*100))